In [ ]:
import cv2
import numpy as np
import serial
import time
import threading
import random
 

from paho.mqtt import client as mqtt_client
broker = 'broker.hivemq.com'
port = 1883
client_id = f'python-mqtt-{random.randint(0, 1000)}'

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def on_message(client, userdata, msg):
   temp = msg.payload.decode()
   f = open("temp.dat", "w")
   f.write(temp)
   f.close()

client = connect_mqtt()
client.subscribe("tx")
client.on_message = on_message
client.loop_start()
time.sleep(3)

f=open("temp.dat","r")
temp = f.read()
f.close()
print('Current Temperture = ' , temp)
temp = int(float (temp))

stamp1 = time.perf_counter()
stamp2= time.perf_counter()
deadstate = time.perf_counter()

print("mqtt client id = ",  client)
# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
#output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading image
cap = cv2.VideoCapture(1)

font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0
startClock = time.perf_counter()
while True:
    _, frame = cap.read()
    frame_id += 1

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00395, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)

    deadsecs = time.perf_counter() - deadstate
    if deadsecs >= 30:
        client.publish("rx", "b")
        client.publish("rx", "d")
        deadstate = time.perf_counter()
        
        
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            #print(label)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            print("{} => {}-{}".format(label,x,y))
            label = label.lower().strip()
            
            if label == "person":
                
                cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 3, color, 3)
                
                secs = time.perf_counter() - stamp1
                if x<=150 and temp >= 28:
                    if secs >= 10:
                        stamp1 = time.perf_counter()
                        print("level-2")
                        client.publish("rx", "a")
                        
                secs = time.perf_counter() - stamp2
                if x>=160 and temp>=28:
                    if secs >=10:
                        stamp2 = time.perf_counter()
                        print("level-3")
                        client.publish("rx", "c")
                        
               
                       
         
    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)
   
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()